In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from matplotlib.backends.backend_pdf import PdfPages

In [48]:
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
#%autoreload now

In [50]:
# Local modules
from db import getData
from pd_helpers import get_modified_isd_df, get_filtered_by_time, get_filtered_by_listed_time_and_quant, get_filtered_by_sellers, get_filtered_by_min_3_price_avg
from pd_styling import set_caption

In [51]:
# Get data
df = getData("SELECT t.* FROM isd_detailed_daily t")

In [52]:
df.sample(5)

,item_url_name_rank,date,name,rank,number_of_sellers,quantity_available,mean_price,median_price,min_price,min_3_price_avg,...,warframe,melee_weapon,mod,arcane,archwing,ducats,skin,lens,riven,misc
17379,chilling_reload_0,2023-03-28 04:00:00+00:00,Chilling Reload,0.0,14.000000,45.000000,4.593333,4.833333,3.333333,4.166667,...,False,False,True,False,False,0,False,False,False,False
80434,steady_hands_0,2023-03-20 04:00:00+00:00,Steady Hands,0.0,7.333333,27.666667,3.340000,3.666667,2.333333,3.000000,...,False,False,True,False,False,0,False,False,False,False
38619,infectious_bite_3,2023-03-23 04:00:00+00:00,Infectious Bite,3.0,1.000000,1.000000,124.000000,124.000000,124.000000,124.000000,...,False,False,True,False,False,0,False,False,False,False
66903,primed_smite_grineer_0,2023-03-22 04:00:00+00:00,Primed Smite Grineer,0.0,7.666667,27.000000,59.360000,60.000000,51.000000,56.000000,...,False,False,True,False,False,0,False,False,False,False
54998,neo_d1_relic,2023-03-24 04:00:00+00:00,Neo D1 Relic,NaN,1.000000,4.000000,20.000000,20.000000,20.000000,20.000000,...,False,False,False,False,False,0,False,False,False,False


In [53]:
df = get_modified_isd_df(df)

In [56]:
df.columns

Index(['item_url_name_rank', 'date', 'name', 'rank', 'set',
       'number_of_sellers', 'quantity_available', 'mean_price', 'median_price',
       'min_price', 'min_3_price_avg', 'avg_listed_time_new_3'],
      dtype='object')

In [61]:
# These are the columns we can perform aggregations on 
agg_cols = df.loc[0,"number_of_sellers":"avg_listed_time_new_3"].index.to_list()
agg_cols

['number_of_sellers',
 'quantity_available',
 'mean_price',
 'median_price',
 'min_price',
 'min_3_price_avg',
 'avg_listed_time_new_3']

In [54]:
sc_df = (
    pd
    .read_feather("set_components.ftr")
    .set_index("item_url_name")
)
sc_df.sample(3)

,set_root_url_name,quantity,item_id,set_root_id,set_quantity
item_url_name,,,,,
scourge_prime_blueprint,scourge_prime_set,1,61bb64fe3132ff00482b5fd6,61bb64fe3132ff00482b5fcf,4
corvas_receiver,corvas_set,1,578d33bf34003bbbb9e20922,582b0f890af25410b099f7ff,3
ember_prime_blueprint,ember_prime_set,1,54a73e65e779893a797fff81,56783f24cbfa8f0432dd899a,4


In [69]:
# Get all sets that meet certain criteria
sets_df = (
    get_filtered_by_time(df[df["set"] == True], days=3).groupby("item_url_name_rank").agg(dict.fromkeys(agg_cols, "median"))
    .pipe(get_filtered_by_min_3_price_avg, min_price=50)
    .pipe(get_filtered_by_sellers, min_sellers=2)
    .pipe(get_filtered_by_listed_time_and_quant, max_listed=500, min_quant=2)
    )

sets_indices = sets_df.index.values

In [70]:
sets_df

,number_of_sellers,quantity_available,mean_price,median_price,min_price,min_3_price_avg,avg_listed_time_new_3
item_url_name_rank,,,,,,,
afuris_prime_set,12.3,14.7,83.4,83.8,74.3,77.7,2.8
akbolto_prime_set,11.7,15.0,84.8,85.0,73.3,77.7,55.8
akjagara_prime_set,5.5,15.0,93.3,95.0,89.5,90.1,227.6
aksomati_prime_set,10.0,15.7,69.8,70.0,64.5,68.1,195.1
akstiletto_prime_set,7.0,17.5,126.5,124.2,108.3,114.7,118.4
...,...,...,...,...,...,...,...
volt_prime_set,23.0,65.7,103.5,100.0,98.5,101.3,18.2
wolf_sledge_set,4.5,7.0,201.5,201.3,192.0,200.6,133.3
wukong_prime_set,25.3,61.0,82.7,82.5,72.5,77.7,2.6


In [71]:
# The result df for Set-Component price difference
sc_diff_df = (
      pd
      .DataFrame(index=sets_df.index, columns=["comp_total", "set_total", "quantity", "sc_diff", "comp_listed_time", "set_listed_time"])
    )

# get a time filtered df
dff = get_filtered_by_time(df, days=7).groupby("item_url_name_rank").agg(dict.fromkeys(agg_cols, "median"))

for set_name in sets_indices:
  set_components = sc_df[sc_df["set_root_url_name"] == set_name][["quantity"]]
  set_components = set_components.assign(
      min_3_price_avg=lambda x: dff.loc[x.index,"min_3_price_avg"] * x.quantity,
      listed_time=lambda x: dff.loc[x.index,"avg_listed_time_new_3"]
  )
  sc_diff_df.loc[set_name] = [
      set_components["min_3_price_avg"].sum(),
      sets_df.loc[set_name,"min_3_price_avg"],
      set_components["quantity"].sum(),
      np.nan,
      round(set_components["listed_time"].mean(),1),
      dff.loc[set_name, "avg_listed_time_new_3"]
      ]
  sc_diff_df.loc[set_name, "sc_diff"] = sc_diff_df.loc[set_name, "set_total"] - sc_diff_df.loc[set_name, "comp_total"]

In [72]:
def style_worthy(v, props=''):
    return np.where(v > 30, props, "")

In [73]:
s = (
    sc_diff_df[sc_diff_df["quantity"] > 0]
     .sort_values("sc_diff", ascending=False)
     .head(10)
     .round(1)
     .style
     #.apply(style_worthy, props='color:green;', axis=0, subset="sc_diff")
     .background_gradient(cmap="GnBu_r", subset="sc_diff", low=0.7, high=0.3)
     .background_gradient(cmap="GnBu", subset="set_listed_time", low=0.2, high=.8)
)
set_caption(s, "Price differences between the total cost of components and the cost of the set.", loc="top")
s

,comp_total,set_total,quantity,sc_diff,comp_listed_time,set_listed_time
item_url_name_rank,,,,,,
arum_spinosa_set,110.800000,143.100000,4,32.300000,271.700000,42.100000
nami_skyla_prime_set,113.600000,143.800000,5,30.200000,38.100000,155.700000
braton_vandal_set,241.700000,271.200000,4,29.500000,414.100000,471.300000
carmine_penta_set,53.200000,80.700000,4,27.500000,294.200000,305.500000
shade_prime_set,47.100000,71.500000,4,24.400000,2.400000,1.300000
vauban_prime_set,114.100000,138.300000,4,24.200000,21.400000,30.000000
akstiletto_prime_set,90.600000,114.700000,6,24.100000,89.500000,118.400000
mag_prime_set,136.300000,158.000000,4,21.700000,37.700000,37.200000
soma_prime_set,67.700000,87.600000,4,19.900000,49.400000,88.300000
